<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/%E9%93%BE%E6%8E%A5%E8%BE%93%E5%87%BA%E7%9A%84%E5%A4%9A%E7%A7%8D%E5%BD%A2%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet  langchain-google-genai pillow
!pip install langchain langchain_community
!pip install --upgrade httpx httpx-sse PyJWT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00


In [2]:
import os
import getpass

api_key = os.getenv("ZHIPUAI_API_KEY")
if api_key is None:
  os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

··········


In [3]:
from langchain_community.chat_models import ChatZhipuAI


from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.chains import ConversationChain

llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

In [5]:
# 使用Pydantic创建数据格式
from pydantic import BaseModel, Field
from typing import List

class Flower(BaseModel):
    name: str = Field(description="name of a flower")
    colors: List[str] = Field(description="the colors of this flower")


In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
template = "{flower}的花语是？"
prompt = PromptTemplate.from_template(template)

In [12]:
# 初始化LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)
llm_chain.invoke("玫瑰")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'flower': '玫瑰',
 'text': '玫瑰花语丰富多彩，不同颜色、数量的玫瑰代表着不同的含义和情感。\n\n颜色的花语如下：\n- 红玫瑰代表热情、热爱着你，象征着我爱你、热恋，希望与你泛起激情的爱。\n- 蓝玫瑰象征着敦厚、善良，也代表浪漫的爱情和美丽的梦想。\n- 粉红玫瑰表达感动、爱的宣言、铭记于心，代表着初恋，喜欢你那灿烂的笑容。\n- 白玫瑰象征天真、纯洁、尊敬和谦卑。\n- 黄玫瑰有多种含义，可以表示不贞、嫉妒，也可以是欢乐、高兴、道歉。\n- 紫玫瑰代表忧郁、梦幻，爱做梦。\n- 橙玫瑰表示羞怯，献给你一份神秘的爱。\n- 绿玫瑰代表纯真简朴、青春长驻，我只钟情你一个。\n\n数量的花语包括：\n- 1朵玫瑰代表你是我的唯一。\n- 2朵玫瑰表示世界上只有你和我。\n- 3朵玫瑰代表“我爱你”（I love you）。\n- 4朵玫瑰象征誓言和承诺。\n- 5朵玫瑰表示无悔。\n- 6朵玫瑰代表顺心如意、顺利、永结同心。\n\n此外，还有其他数量的玫瑰代表不同的意义，如99朵代表天长地久，100朵代表百分之百的爱，等等。\n\n玫瑰花以其美丽的外观和丰富的花语，成为表达情感和传递爱意的绝佳选择。'}

In [14]:
input_list = [
    { 'flower': "玫瑰", 'season': "夏季" },
    { 'flower': "康乃馨", 'season': "冬季" },
    { 'flower': "郁金香", 'season': "春季" },
]
print(llm_chain.run(input_list))

每种花都有其特定的花语，通常与它们的特性、开花季节以及文化寓意有关。以下是您提供的三种花及其对应季节的花语：

1. 玫瑰（夏季）：玫瑰象征着爱情和热情。不同颜色的玫瑰还有不同的含义，比如：
   - 红玫瑰代表热烈的爱情。
   - 粉玫瑰代表初恋或美好的祝愿。
   - 白玫瑰代表纯洁或尊敬。
   - 黄玫瑰代表友谊或关怀。

   夏季的玫瑰花语通常与生命力和热情有关。

2. 康乃馨（冬季）：康乃馨的花语是爱与魅力，它也代表着母爱和感恩。不同颜色的康乃馨也有不同的含义：
   - 红康乃馨代表母爱。
   - 粉康乃馨代表我永远不会忘记你。
   - 白康乃馨代表纯洁的爱。

   冬季的康乃馨花语可能与温暖、亲情和感恩有关。

3. 郁金香（春季）：郁金香的花语是完美的爱情和名誉。它也代表着永恒的幸福和快乐。不同颜色的郁金香也有不同的含义：
   - 红郁金香代表真爱。
   - 紫郁金香代表尊贵和永恒。
   - 黄郁金香代表快乐和阳光。

   春季的郁金香花语通常与新生、希望和完美的爱情有关。

请注意，花语会因地区和文化背景的不同而有所差异，上述花语是比较通用的解释。


In [15]:
input_list = [
    { 'flower': "玫瑰", 'season': "夏季" },
    { 'flower': "康乃馨", 'season': "冬季" },
    { 'flower': "郁金香", 'season': "春季" },
]
print(llm_chain.apply(input_list))

[{'text': '玫瑰花作为情感表达的载体，其花语丰富而多样，不同颜色和数量的玫瑰代表着不同的含义。\n\n首先，红色玫瑰象征着爱情、热情和热恋，通常用来表达“我爱你”，是情侣间表达爱意的首选。\n\n粉色玫瑰则代表着初恋、感动以及温馨的爱，它传递着一种温柔的情感，适合向心仪的对象表白。\n\n蓝色玫瑰寓意着神秘、珍贵以及不可能实现的事，因其稀有而显得独特，常用来表达对某人的深情和唯一。\n\n白色玫瑰则代表着纯洁、天真和尊敬，它所传递的是一种清新脱俗的爱，也适合作为对长辈的敬意表达。\n\n黄色玫瑰有着多样的含义，可以是友情的祝福，也可以是道歉，但需注意，在某些情境下它可能代表着失恋或分手。\n\n紫色玫瑰则代表着忧郁、梦幻和浪漫，常用来表达一种深沉而神秘的情感。\n\n至于玫瑰的数量，也有其特定的含义：\n- 一朵玫瑰代表“你是我的唯一”。\n- 两朵玫瑰象征“只有你和我”。\n- 三朵玫瑰表示“我爱你”。\n- 四朵玫瑰代表“誓言与承诺”。\n- 五朵玫瑰意味着“无悔的爱”。\n- 六朵玫瑰则寓意“一切顺利”或“永结同心”。\n\n玫瑰花语是人们赋予花的文化意义，通过赠送不同颜色和数量的玫瑰，可以传递各种情感和信息。'}, {'text': '康乃馨的花语十分丰富，它代表着热情、魅力、温馨的爱、真情、母亲我爱你、慈祥、不求代价的母爱、宽容、浓郁的亲情、清纯的爱慕之情、热恋、热心、伤心与懊悔、伟大、神圣、慰问、心灵的相通、真挚、走运和思念等美好寓意。由于康乃馨的花语多样且具有深厚的情感象征，它常被用作表达对母亲的爱和感激之情的母亲节礼物。此外，不同颜色的康乃馨还有各自的特定含义，比如：\n\n- 红色康乃馨：代表爱情、热情和崇拜。\n- 粉色康乃馨：通常与母爱、友情和感激联系在一起。\n- 白色康乃馨：象征纯洁、纯真和无邪。\n- 黄色康乃馨：表示友情和欢乐。\n- 橙色康乃馨：代表欢乐和兴奋。\n- 紫色康乃馨：通常与权力、尊严和尊敬相关联。\n\n康乃馨的这些花语使得它在多种场合都能作为合适的礼物，不仅限于母亲节，还包括毕业、三八节等，表达对亲人或朋友的美好祝愿和深厚情感。'}, {'text': '郁金香的花语十分丰富，它代表着爱、慈善、名誉、美丽、祝福和永恒等多种美好的寓意。不同颜色的郁金香还有各自独特的花语：\n\n- 红色郁金香代表热烈的爱意、爱

In [16]:
print(llm_chain.predict(flower="玫瑰", season="夏季"))

玫瑰花语丰富多样，长久以来它都是美丽和爱情的象征。不同颜色和数量的玫瑰花代表着不同的含义：

- 颜色花语：
  - 红玫瑰：代表热情、热爱、我爱你、热恋，希望与你共浴爱河。
  - 蓝玫瑰：象征敦厚、善良。
  - 粉红玫瑰：表示感动、爱的宣言、铭记于心、初恋，喜欢那灿烂的笑容。
  - 白玫瑰：代表天真、纯洁、尊敬、谦卑。
  - 黄玫瑰：含义有不贞、嫉妒，也有欢乐、高兴、道歉之意。
  - 紫玫瑰：代表忧郁、梦幻、爱做梦。
  - 橙玫瑰：象征羞怯、献给你一份神秘的爱。
  - 绿玫瑰：代表纯真简朴、青春长驻，我只钟情你一个。

- 数量花语：
  - 1朵：你是我的唯一。
  - 2朵：世界上只有你和我。
  - 3朵：代表“我爱你”（I love you）。
  - 4朵：象征誓言、承诺。
  - 5朵：表示无悔。
  - 6朵：代表顺心如意、顺利、永结同心。

玫瑰花因其美丽的外观和丰富的花语，成为情侣间表达爱意的理想选择。在特定节日或纪念日，赠送不同颜色和数量的玫瑰花，可以传达各种美好的情感和愿望。


In [17]:
print(llm_chain.generate(input_list))

generations=[[ChatGeneration(text='玫瑰花语丰富多样，不同颜色的玫瑰代表着不同的含义。以下是一些常见的玫瑰花语：\n\n- 红玫瑰：爱情、热情、我爱你\n- 粉玫瑰：初恋、感谢、温柔的爱\n- 白玫瑰：纯洁、尊敬、我值得你\n- 黄玫瑰：友谊、关怀、快乐\n- 橙玫瑰：激情、热情、渴望\n- 紫玫瑰：魅力、神秘、爱情无望\n- 蓝玫瑰：神秘、不可能的爱、渴望得到你\n\n当然，这些花语并非固定不变，不同文化和地区可能有不同的解读。送花时，人们还可以根据自己的意图赋予玫瑰特定的含义。', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='玫瑰花语丰富多样，不同颜色的玫瑰代表着不同的含义。以下是一些常见的玫瑰花语：\n\n- 红玫瑰：爱情、热情、我爱你\n- 粉玫瑰：初恋、感谢、温柔的爱\n- 白玫瑰：纯洁、尊敬、我值得你\n- 黄玫瑰：友谊、关怀、快乐\n- 橙玫瑰：激情、热情、渴望\n- 紫玫瑰：魅力、神秘、爱情无望\n- 蓝玫瑰：神秘、不可能的爱、渴望得到你\n\n当然，这些花语并非固定不变，不同文化和地区可能有不同的解读。送花时，人们还可以根据自己的意图赋予玫瑰特定的含义。', response_metadata={'token_usage': {'completion_tokens': 137, 'prompt_tokens': 10, 'total_tokens': 147}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-ef753b16-d574-4731-aa94-49ada238f690-0'))], [ChatGeneration(text='康乃馨的花语十分丰富，它代表着热情、魅力、温馨的爱、真情、母亲我爱你、慈祥、不求代价的母爱、宽容、浓郁的亲情、清纯的爱慕之情、热恋、热心、伤心与懊悔、伟大、神圣等多种美好的寓意。由于康乃馨的花色多样，不同的颜色还代表着不同的含义：\n\n- 红色康乃馨：通常象征着爱情、热情和崇拜。\n- 粉色康乃馨：表达了母爱、友情和感激之情。\n- 白色康乃馨：代表着纯洁、纯真、无邪和忠诚。\n- 黄色康乃馨：通常象征着友情和欢乐。\